# Parameters

In [29]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [30]:
%%bash

mkdir "/content/gdrive/MyDrive/6864_project/"
mkdir "/content/gdrive/MyDrive/6864_project/model_comment"


mkdir: cannot create directory ‘/content/gdrive/MyDrive/6864_project/’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/6864_project/model_comment’: File exists


In [31]:
#destination_folder_processing = "/content/gdrive/MyDrive/6864_project/train_data/max_mean_topic"

first_n_words = 512

# Libraries

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Preprocessing

In [33]:
def trim_string(x):

    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])

    return x

In [34]:
data = pd.read_csv("gdrive/MyDrive/6864_project/data/infertility/processed_posts.csv")
data_ttc = pd.read_csv("gdrive/MyDrive/6864_project/data/ttcafterloss/processed_posts.csv")
dfs = [data, data_ttc]
data = pd.concat(dfs)
topic = pd.read_csv("gdrive/MyDrive/6864_project/data/10_all_posts_with_comment_topics.csv")
data = data.merge(topic[["id","max_mean_topic","mode_max_topic"]], on='id')
data = data[~data.max_mean_topic.isna()]

def get_bin(x):
  if x < 4:
    return 0
  if x < 8:
    return 1
  if x < 11:
    return 2
  if x < 16:
    return 3
  if x < 25:
    return 4
  return 5

def get_bin_upvote(x):
  if x < .86:
    return 0
  if x < 1:
    return 1
  return 2

data["comments_bin"] = data["num_comments"].apply(lambda x: get_bin(x))
data["upvotes_bin"] = data["upvote_ratio"].apply(lambda x: get_bin_upvote(x))

In [35]:
#Handling long data inputs

#split into chunks of 200 words with 50 words overlapped
def get_split(text1):
  
  text1 = "".join(text1.split("]"))
  text1 = "".join(text1.split("["))

  text1 = " ".join(text1.split(","))

  text1 = text1.replace("'", "")

  return text1[:first_n_words]

data = data[data.processed_text.str.len() > 10]
data["processed_text"] = data["processed_title"] + "," + data["processed_text"]
data["text_split"] = data["processed_text"].apply(get_split)
data.head()

,Unnamed: 0,id,created_utc,author,author_fullname,author_flair_text,url,title,selftext,upvote_ratio,score,num_comments,data_split,processed_text,processed_title,max_mean_topic,mode_max_topic,comments_bin,upvotes_bin,text_split
0,0,2t72mf,1.421865e+09,nhmejia,t2_ivw2a,Hard to get pregnant after a hysto.,https://www.reddit.com/r/infertility/comments/...,*TRIGGER WARNING* Not that funny...,I've been doing really well staying away from ...,0.80,8,25,val,"[['trigger', 'warn', 'not', 'that', 'funny']],...","[['trigger', 'warn', 'not', 'that', 'funny']]",4.0,4.0,5,0,trigger warn not that funny I be do rea...
1,1,2t6u8l,1.421862e+09,septicidal,t2_6h9ib,PCOS,https://www.reddit.com/r/infertility/comments/...,Update on planning for a new cycle,I know a bunch of people saw my update in the ...,1.00,7,8,train,"[['update', 'on', 'plan', 'for', 'a', 'new', '...","[['update', 'on', 'plan', 'for', 'a', 'new', '...",4.0,4.0,2,2,update on plan for a new cycle I know ...
2,3,2t57np,1.421820e+09,PoisonRose23,t2_j2zbu,NaN,https://www.reddit.com/r/infertility/comments/...,Decision about fermara! I need advice!!!,Long post warning: So I have a decision to mak...,1.00,1,6,train,"[['decision', 'about', 'fermara', 'I', 'need',...","[['decision', 'about', 'fermara', 'I', 'need',...",6.0,6.0,1,2,decision about fermara I need advice long...
3,5,2t45rt,1.421801e+09,Pamzella,t2_aifrd,41 MFI & DOR 1MC mult IUIs,https://www.reddit.com/r/infertility/comments/...,"""Shared risk"" clinics","Please educate me, someone asked me about this...",1.00,7,17,train,"[['share', 'risk', 'clinic']],[['please', 'edu...","[['share', 'risk', 'clinic']]",6.0,6.0,4,2,share risk clinic please educate I someon...
4,6,2t3x6r,1.421797e+09,fumbellina,t2_iibv6,NaN,https://www.reddit.com/r/infertility/comments/...,FET anxiety,"My FET is scheduled for next week, and I'm fee...",0.81,3,6,train,"[['fet', 'anxiety']],[['my', 'fet', 'be', 'sch...","[['fet', 'anxiety']]",6.0,6.0,1,0,fet anxiety my fet be schedule for next ...


In [36]:
# # Read raw data
# df_raw = data

# # Prepare columns
# df_raw['label'] = df_raw["max_mean_topic"]
# df_raw['titletext'] = df_raw["text_split"]
# df_raw = df_raw.reindex(columns=['label','titletext',"id","data_split"])

# # Drop rows with empty text
# df_raw.drop( df_raw[df_raw.titletext.str.len() < 5].index, inplace=True)

# # Trim text and titletext to first_n_words
# # df_raw['text'] = df_raw['text'].apply(trim_string)
# df_raw['titletext'] = df_raw['titletext'].apply(trim_string) 

# df_train = df_raw[df_raw.data_split == "train"]
# df_train = df_raw[df_raw.data_split == "val"]
# df_train = df_raw[df_raw.data_split == "test"]

# # # Split according to label
# # df_real = df_raw[df_raw['label'] == 0]
# # df_fake = df_raw[df_raw['label'] == 1]

# # Train-test split
# #df_train, df_test = train_test_split(df_raw, train_size = train_test_ratio, random_state = 1)
# # df_fake_full_train, df_fake_test = train_test_split(df_fake, train_size = train_test_ratio, random_state = 1)

# # Train-valid split
# #df_train, df_valid = train_test_split(df_train, train_size = train_valid_ratio, random_state = 1)
# # df_fake_train, df_fake_valid = train_test_split(df_fake_full_train, train_size = train_valid_ratio, random_state = 1)

# # Concatenate splits of different labels
# # df_train = pd.concat([df_real_train, df_fake_train], ignore_index=True, sort=False)
# # df_valid = pd.concat([df_real_valid, df_fake_valid], ignore_index=True, sort=False)
# # df_test = pd.concat([df_real_test, df_fake_test], ignore_index=True, sort=False)

# # Write preprocessed data
# df_train.to_csv(destination_folder_processing + '/train.csv', index=False)
# df_valid.to_csv(destination_folder_processing + '/valid.csv', index=False)
# df_test.to_csv(destination_folder_processing + '/test.csv', index=False)

In [37]:
!pip install transformers

In [38]:
# !pip install torch==1.6 torchtext==0.7

In [39]:
# !pip install -U torchtext==0.9.0

In [40]:
# !pip install -U torch==1.8.1

In [41]:
!pip list | grep torch

torch                         1.8.1+cu101   
torchsummary                  1.5.1         
torchtext                     0.9.1         
torchvision                   0.9.1+cu101   


In [42]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

# from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models
import transformers

import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [43]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device="cpu"
print(device)

cuda:0


In [44]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [45]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, df):
        'Initialization'
        self.df = df

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

  def __getitem__(self, index):

        # Load data and get label
        row = self.df.iloc[index]
        encode = tokenizer.encode_plus(row.text_split, add_special_tokens=True, max_length=512, padding='max_length', return_attention_mask=True,return_tensors='pt')
        input_ids = torch.cat([x.float() for x in encode["input_ids"]], dim=0).to(torch.int64)
        attention_mask = torch.cat([x.float() for x in encode["attention_mask"]], dim=0)
        #label = torch.tensor(row.num_comments).to(torch.float)
        label = torch.tensor(row.comments_bin).to(torch.long)
        #label = torch.tensor(row.upvote_ratio).to(torch.float)
        #label = torch.tensor(row.mode_max_topic).to(torch.long)
        id = row.id
        sample = {"input_ids":input_ids, "attention_mask":attention_mask, "label":label, "id":id}
        #print(sample["label"], row.num_comments)
        return sample


In [46]:
BATCH_SIZE= 16
def custom_collate(batch):
  return {"input_ids":nn.utils.rnn.pad_sequence(stack_vals(batch,"input_ids"),batch_first=True), "attention_mask":nn.utils.rnn.pad_sequence(stack_vals(batch,"attention_mask"),batch_first=True), "label":torch.tensor(stack_vals(batch,"label"))}

train_df = data[data.data_split=="train"]
train_ds = Dataset(train_df)
#train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,collate_fn=custom_collate)
train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

val_df = data[data.data_split=="val"]
val_ds = Dataset(val_df)
#val_ldr = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True,collate_fn=custom_collate)
val_ldr = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True)

test_df = data[data.data_split=="test"]
test_ds = Dataset(test_df)
#val_ldr = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True,collate_fn=custom_collate)
test_ldr = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

In [47]:
# # Model parameter
# MAX_SEQ_LEN = 128
# PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
# UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# # Fields

# label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
# text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
#                    fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
# fields = [('label', label_field),('titletext', text_field)]

# # TabularDataset

# train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
#                                            test='test.csv', format='CSV', fields=fields, skip_header=True)

# # Iterators

# train_iter = BucketIterator(train, batch_size=8, sort_key=lambda x: len(x.titletext),
#                             device=device, train=True, sort=True, sort_within_batch=True)
# valid_iter = BucketIterator(valid, batch_size=8, sort_key=lambda x: len(x.titletext),
#                             device=device, train=True, sort=True, sort_within_batch=True)
# test_iter = Iterator(test, batch_size=8, device=device, train=False, shuffle=False, sort=False)

In [58]:
class BERT(nn.Module):

    def __init__(self, config):
        super(BERT, self).__init__()
        self.encoder = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config = config)


    def forward(self, text, label):
        loss, text_fea,hidden = self.encoder(input_ids=text, labels=label,output_hidden_states=True)[:3]
        #print(text_fea.shape)

        return loss, text_fea, hidden[-1]

In [59]:
# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [60]:
# Training Function

def train(model,
          optimizer,
          criterion = nn.CrossEntropyLoss(),
          train_loader = train_ldr,
          valid_loader = val_ldr,
          num_epochs = 5,
          eval_every = len(train_ldr) // 2,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for index, batch in enumerate(train_loader):
            labels = batch["label"].type(torch.LongTensor)           
            labels = labels.to(device)
            titletext = batch["input_ids"].type(torch.LongTensor)  
            titletext = titletext.to(device)
            mask = batch["attention_mask"].type(torch.LongTensor)  
            mask = titletext.to(device)

            output = model(titletext, label=labels)
            loss, _ = output

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            if index % 20 == 1:
              print(f'At step {index *16}, the loss = {loss}, {running_loss / index}')
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    

                    # validation loop
                    for batch in valid_loader:
                        labels = batch["label"].type(torch.LongTensor)           
                        labels = labels.to(device)
                        titletext = batch["input_ids"].type(torch.LongTensor)  
                        titletext = titletext.to(device)
                        mask = batch["attention_mask"].type(torch.LongTensor)  
                        mask = titletext.to(device)

                        output = model(titletext, label=labels)
                        loss, _ = output
                        
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

In [61]:

destination_folder = "/content/gdrive/MyDrive/6864_project/model_comment"

from transformers import DistilBertConfig
config = DistilBertConfig.from_pretrained('distilbert-base-uncased',num_labels=6)

In [62]:
#DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config = config)

In [63]:
import os
print(device)
#device= "cpu"
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model = BERT(config).to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

#train(model=model, optimizer=optimizer)

cuda:0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [55]:
load_checkpoint(destination_folder + '/' + 'model.pt', model)
from sklearn.metrics import f1_score,  accuracy_score

model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []   
    valid_running_loss = 0               

    # validation loop
    for batch in test_ldr:
        labels = batch["label"].type(torch.LongTensor)           
        labels = labels.to(device)
        titletext = batch["input_ids"].type(torch.LongTensor)  
        titletext = titletext.to(device)

        output = model(titletext, label=labels)
        loss, logits = output
        bin = torch.argmax(logits,dim=1)
        #print(batch["label"], bin, bin.shape)
        y_true.append(batch["label"])
        y_pred.append(bin)
      
        valid_running_loss += loss.item()

# evaluation

from sklearn.metrics import f1_score,  accuracy_score
import numpy as np
print(valid_running_loss / len(test_ldr))
y_true = torch.cat(y_true).detach().cpu().numpy().flatten()
y_pred = torch.cat(y_pred).detach().cpu().numpy().flatten()
print(y_true, y_pred)
print("accuracy", accuracy_score(y_true, y_pred, average="micro"))
print("f1", f1_score(y_true, y_pred, average="micro"))


Model loaded from <== /content/gdrive/MyDrive/6864_project/model_comment/model.pt
1.649937872974961
[1 2 2 ... 1 1 1] [1 1 1 ... 1 4 1]


TypeError: ignored

In [28]:
from sklearn.metrics import f1_score,  accuracy_score
import numpy as np
print(valid_running_loss / len(test_ldr))
y_true = torch.cat(y_true).detach().cpu().numpy().flatten()
y_pred = torch.cat(y_pred).detach().cpu().numpy().flatten()
print(y_true, y_pred)
print("accuracy", accuracy_score(y_true, y_pred,average="micro"))
print("f1", f1_score(y_true, y_pred,average="micro"))

1.699371353343681
[1 4 3 ... 3 3 3] [1 3 1 ... 0 4 4]


TypeError: ignored

In [56]:
np.savetxt("/content/gdrive/MyDrive/6864_project/pred_comments.txt", np.array([y_true,y_pred]))

In [64]:
load_checkpoint(destination_folder + '/' + 'model.pt', model)
model.eval()
outputs= []
for index, batch in enumerate(train_ldr):
    labels = batch["label"].type(torch.LongTensor)           
    labels = labels.to(device)
    titletext = batch["input_ids"].type(torch.LongTensor)  
    titletext = titletext.to(device)

    output = model(titletext, label=labels)
    loss, logits, hidden = output
    embed = hidden[:,0,:]
    if index % 20 == 1:
      print(index*16)
    #print(embed.shape)
    for i in range(16):
      latest_output = {"id": batch["id"][i],"label": batch["label"].numpy()[i], "prediction":torch.argmax(logits,dim=1).detach().cpu().numpy()[i],"embedding":embed.detach().cpu().numpy()[i] }
      #print(embed.detach().cpu().numpy()[i].shape)
      outputs.append(latest_output)
    #print(latest_output)

Model loaded from <== /content/gdrive/MyDrive/6864_project/model_comment/model.pt
16
336
656
976
1296
1616
1936
2256
2576
2896
3216
3536
3856
4176
4496
4816
5136
5456
5776
6096
6416
6736
7056
7376
7696
8016
8336
8656
8976
9296
9616
9936
10256
10576
10896
11216
11536
11856
12176
12496
12816


IndexError: ignored

In [65]:
df = pd.DataFrame(outputs)

In [66]:
df.to_csv("/content/gdrive/MyDrive/6864_project/bert_seq_class_comments.csv")

In [67]:
df

,id,label,prediction,embedding
0,d5k1p9,3,1,"[-0.17213681, 0.12055801, 0.008012729, -0.1584..."
1,fiaoe6,0,1,"[-0.32413787, -0.09687651, 0.049001757, -0.380..."
2,3k4gvy,1,0,"[-0.1798693, -0.10688318, -0.08189541, -1.0396..."
3,8orpvf,2,4,"[0.026685385, -0.12186421, 0.05015227, 0.04255..."
4,e9wo1p,2,4,"[-0.27250776, -0.56289303, 0.1940978, 0.183885..."
...,...,...,...,...
12889,ekx90i,2,1,"[-0.47269633, -0.516186, 0.20204246, -0.143793..."
12890,88w2e9,1,1,"[-0.54807967, -0.3665095, 0.059043285, -0.1831..."
12891,avdhev,1,1,"[-0.4859991, -0.2646182, 0.3061617, -0.0750821..."
12892,f0x78o,0,1,"[-0.3363064, -0.118753195, 0.27506822, -0.4199..."
